# Facet spaces and hybrid methods (Unit 2.7)

Mixed methods for second order problems lead to saddle point problems, and indefinite matrices. By hybridization one obtains a positive definite system again. It's structure is similar to the non-conforming $\mathcal{P}^1$ method, but hybridization works for any order. See text-book by Brezzi and Fortin.

One skips the normal-continuity of the $H(div)$ variable, and reinforces it by a Lagrange parameter. This leads to the following discrete system:

Find $\sigma, u, \hat{u} \in \Sigma_h \times V_h \times F_h$:

$$\int \lambda^{-1} \sigma \tau + \sum_T \int_T \text{ div } \tau u + \sum_F \int_F [\tau n] \hat{u} = 0 \ \ \forall \tau \in \Sigma_h$$

$$\int \text{div } \sigma v = \int fv \ \ \forall v \in V_h$$

$$\int [\sigma n] \hat{v} = \int_{\Gamma_n} g \hat{v} \ \ \forall \hat{v} \in F_h $$ 

where $\Sigma_h$ is a discontinuous $H(div)$ finite element space, $V_h$ a sub-space of $L_2$, and $F_h$ consists of polynomials on every edge.

### Notes: 
I think $F_h$ stands for Facet

What is the problem we're trying to solve here?  I think it's exactly the diffusion problem in Mixed. The main difference is that now we have $\hat{u}$ (edge polynomials) and $\sigma$ can be discontinous.  Open the Mixed notebook and compare!

We can use the 'discontinuous' keyword for a finite element space to specify whether a space is discontinuous.  Read on...

In [1]:
from netgen.geom2d import unit_square
from ngsolve import *
import netgen.gui
%gui tk
mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))


same example as in 'mixed':

In [2]:
source = sin(3.14*x)
ud = CoefficientFunction(5)
g = CoefficientFunction([y*(1-y) if bc=="left" else 0 for bc in mesh.GetBoundaries()])
lam = 10

define spaces:

- The discontinuous flag generates an element-wise $H(Div)$-space
- FacetFESpace lives only on facets (i.e. faces in 3D, edges in 2D, points in 1D)

Boundary conditions are now posed for the facet-space

In [3]:
order = 3
V = HDiv(mesh, order=order, discontinuous = True)
Q = L2(mesh, order=order-1)
F = FacetFESpace(mesh, order=order, dirichlet="bottom")
X = FESpace([V,Q,F])
print ("sigmadofs:", X.Range(0))
print ("udofs:    ", X.Range(1))
print ("uhatdofs: ", X.Range(2))


sigmadofs: slice(0, 1120, 1)
udofs:     slice(1120, 1456, 1)
uhatdofs:  slice(1456, 1832, 1)


Assemble forms. The jump-term is rewritten as

$$\sum_F \int_F [\sigma_n] v = \sum_T \int_{\partial T} \sigma_n v$$


Ok, so sum of integrals of jumps on facets equals sum of integrals over element boundaries... 

Is the jump term $[\sigma_n]=\sigma_{n_{left}}−\sigma_{n_{right}}$?

In [8]:
sigma,u,uhat = X.TrialFunction()
tau,v,vhat = X.TestFunction()

condense=True

a = BilinearForm(X, eliminate_internal=condense)
a += SymbolicBFI(1/lam * sigma*tau + div(sigma)*v + div(tau)*u)
n = specialcf.normal(mesh.dim)
a += SymbolicBFI(sigma*n*vhat+tau*n*uhat, element_boundary=True)

# Compare with mixed!
# am = BilinearForm(fesm)
# am += SymbolicBFI(1/lam * sigma * tau + div(sigma)*v + div(tau)* u)
# am.Assemble()

c = Preconditioner(a, "bddc")

f = LinearForm(X)
f += SymbolicLFI(-source*v)
# this gives "cannot evaluate facet inside element" in newer ngsolve
# f += SymbolicLFI(-g*vhat, BND) # Need Trace()
f += SymbolicLFI(-g*vhat.Trace(), BND)

# Compare with mixed!
# fm = LinearForm(fesm)
# fm += SymbolicLFI(-source*v)
# normal = specialcf.normal(mesh.dim)
# fm += SymbolicLFI(ud * (tau.Trace()*normal), BND)
# fm.Assemble()
a.Assemble()
print ("A non-zero", a.mat.nze)

gfu = GridFunction(X)


A non-zero 6880


Solve system. Leave everything to the sparse direct solver, or use CG

In [9]:
f.Assemble()
gfu.components[2].Set(ud, BND)

if condense:
    f.vec.data += a.harmonic_extension_trans * f.vec
    
    solvers.CG(mat=a.mat, pre=c.mat, rhs=f.vec, sol=gfu.vec, initialize=False)
    
    gfu.vec.data += a.harmonic_extension * gfu.vec
    gfu.vec.data += a.inner_solve * f.vec
else:
    r = f.vec.CreateVector()
    r.data = f.vec - a.mat * gfu.vec
    inv = a.mat.Inverse(freedofs=X.FreeDofs())
    gfu.vec.data = inv * r
    


it =  0  err =  136.8191868086319
it =  1  err =  60.516513272990274
it =  2  err =  24.136100089360756
it =  3  err =  12.537226028566534
it =  4  err =  4.6597378335093635
it =  5  err =  1.6826333630097998
it =  6  err =  0.6520593990049843
it =  7  err =  0.2536829638538802
it =  8  err =  0.10105347619782698
it =  9  err =  0.036154331316412006
it =  10  err =  0.02136691657112922
it =  11  err =  0.007693795930838057
it =  12  err =  0.003261934205614721
it =  13  err =  0.001215624309297893
it =  14  err =  0.0005170547168348946
it =  15  err =  0.00018477883556307118
it =  16  err =  6.139392640661932e-05
it =  17  err =  2.3631110433991963e-05
it =  18  err =  8.96379181959564e-06
it =  19  err =  4.210116567505514e-06
it =  20  err =  1.0952387787503712e-06
it =  21  err =  5.00111515102993e-07
it =  22  err =  1.9156387389959438e-07
it =  23  err =  8.848840826450581e-08
it =  24  err =  3.7501229354090414e-08
it =  25  err =  1.273194478384614e-08
it =  26  err =  3.9316364

In [10]:
Draw (gfu.components[0], mesh, "sigma")
Draw (gfu.components[1], mesh, "u")